# 1. Introduction

The objective of this assignment is to perform the numerical calculation of the heat equation in one spatial dimension, defined in Equation 1. This equation describes the diffusion of a passive scalar in a field for any number of spatial dimensions.

$\frac{\partial T}{\partial t} = \frac{\partial^2 T}{\partial x^2} $
**Equation 1** 

The domain of this assignment is a simulated wall of a given thickness originall at a given temperature. The wall has a known diffusion of the heat transfer or temperature difference. In the transient simulation, the sides of the wall are suddenly increased to a given temperature, bringing the temperature of the wall up to some value.

**Table 1**
|   Parameter   |   Value   |
|---------------|-----------|
|   ($L$) Thickness of the simulated wall |   0.3 [$m$] |
|   ($T_0$) Initial uniform temperature of the wall | 100 [$K$] |
|   ($\alpha$) Heat diffusivity |   3e-6 [$m^2/s$]    |
|   ($T_w$) Sudden wall temperature |   300 [$K$]   |
|   ($dx$) Spatial mesh spacing |   0.015 [$m$] |



In [1]:
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt

# Add the directory containing your module to sys.path
module_path = os.path.abspath(os.path.join('..', r"A:\Users\mtthl\Documents\Education\ME5653_CFD\git\me5653_CFD_repo\code\lib" ))
sys.path.append(module_path)

from distributedObjects import *
from distributedFunctions import *


In [2]:
L = 0.300 #[m]
T_0 = 100 #[K]
T_w = 300 #[K]
dx = 0.015 #[m]
alpha = 3e-6 #[m2/s]

x = np.arange( 0 , L+dx , dx )
T = T_0 * np.ones( len( x ) )
T[0] = T_w
T[-1]= T_w


# 2. FTCS


In [3]:
heatEq_FTCS = heatEquation( x , T , ( 0 , 30 * 60 ) , alpha = alpha , dt = 20 )
heatEq_FTCS.solve()


c:\Users\mtthl\.conda\envs\davinci\lib\site-packages\scipy\sparse\_index.py:151: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)
c:\Users\mtthl\.conda\envs\davinci\lib\site-packages\scipy\sparse\_index.py:108: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
A:\Users\mtthl\Documents\Education\ME5653_CFD\git\me5653_CFD_repo\code\lib\distributedObjects.py:446: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  cls.u[i+1,:] = spsr.linalg.spsolve( cls.A , b )


In [4]:
heatEq_FTCS.u

array([[300.        , 100.        , 100.        , ..., 100.        ,
        100.        , 300.        ],
       [300.        , 153.33333333, 100.        , ..., 100.        ,
        153.33333333, 300.        ],
       [300.        , 178.22222222, 114.22222222, ..., 114.22222222,
        178.22222222, 300.        ],
       ...,
       [300.        , 277.38229842, 255.36145873, ..., 255.36145873,
        277.38229842, 300.        ],
       [300.        , 277.54146159, 255.67354357, ..., 255.67354357,
        277.54146159, 300.        ],
       [300.        , 277.69896036, 255.98247571, ..., 255.98247571,
        277.69896036, 300.        ]])